# B. Competition 

MLMaverick Team
- Seulah Lee 300361747
- David Steven Salazar Villegas 300356922
- Ishwinder Singh 300351938

### 1. Load the dataset
- Create a cleaned copy of the DataFrame

In [1]:
import pandas as pd


df = pd.read_csv('bank-sampled-9K.csv',index_col=0)

df_cleaned = df.dropna()
df_cleaned.head()
df_cleaned.shape

(9000, 17)

### Removing Outliers using zscore 

In [2]:

condition = df.poutcome == 'other'
df_cleaned = df.drop(df[condition].index, axis = 0, inplace = False)

df_cleaned[['job','education']] = df_cleaned[['job','education']].replace(['unknown'],'other')
print(df_cleaned.shape)


from scipy.stats import zscore

df_cleaned[['balance']].mean()
df_cleaned[['balance']].mean()

df_cleaned['balance_outliers'] = df_cleaned['balance']
df_cleaned['balance_outliers']= zscore(df_cleaned['balance_outliers'])

condition1 = (df_cleaned['balance_outliers']>3) | (df_cleaned['balance_outliers']<-3 )
df_cleaned = df_cleaned.drop(df_cleaned[condition1].index, axis = 0, inplace = False)
df_cleaned = df_cleaned.drop('balance_outliers', axis=1)
print(df_cleaned.shape)

(8647, 17)
(8524, 17)


### Dropping Duration , merging day and month

In [3]:
df_cleaned.drop('duration',axis=1,inplace=True)

from datetime import datetime as dt

last_date = "18 Nov, 2010"
dt_1 = dt.strptime(last_date, "%d %b, %Y")

df_cleaned['month'] = df_cleaned['month'].apply(lambda x: x.capitalize())
df_cleaned['date'] = df_cleaned['day'].astype(str) + " " + df_cleaned['month'] + ", 2010"

df_cleaned['last_contact'] = df_cleaned['date'].apply(lambda x: (dt_1 - dt.strptime(x, "%d %b, %Y")).days)

df_cleaned.drop(columns=['day', 'date', 'month'], axis=1, inplace=True)

df_cleaned.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,campaign,pdays,previous,poutcome,y,last_contact
0,58,management,married,tertiary,no,2143,yes,no,unknown,1,-1,0,unknown,no,197
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,1,-1,0,unknown,no,197
11,29,admin.,single,secondary,no,390,yes,no,unknown,1,-1,0,unknown,no,197
12,53,technician,married,secondary,no,6,yes,no,unknown,1,-1,0,unknown,no,197
17,57,blue-collar,married,primary,no,52,yes,no,unknown,1,-1,0,unknown,no,197
18,60,retired,married,primary,no,60,yes,no,unknown,1,-1,0,unknown,no,197
24,40,retired,married,primary,no,0,yes,yes,unknown,1,-1,0,unknown,no,197
38,36,admin.,divorced,secondary,no,506,yes,no,unknown,1,-1,0,unknown,no,197
40,44,services,divorced,secondary,no,2586,yes,no,unknown,1,-1,0,unknown,no,197
50,54,blue-collar,married,secondary,no,1291,yes,no,unknown,1,-1,0,unknown,no,197


### Checking datatypes of the columns

In [4]:
df_cleaned.dtypes

age              int64
job             object
marital         object
education       object
default         object
balance          int64
housing         object
loan            object
contact         object
campaign         int64
pdays            int64
previous         int64
poutcome        object
y               object
last_contact     int64
dtype: object

### Arranging categorical columns using dummy values

In [5]:
categorical_columns = df_cleaned.select_dtypes('object').columns
dummy_dfs = []

for column in categorical_columns:
    prefix = column
    dummies = pd.get_dummies(df_cleaned[column], prefix=prefix, drop_first=True)
    dummies.columns = dummies.columns.map(lambda x: x.replace('-', '_'))
    dummies = dummies.astype(int) 
    dummy_dfs.append(dummies)
    df_cleaned.drop(column, axis=1, inplace=True)

df_cleaned = pd.concat([df_cleaned] + dummy_dfs, axis=1)

print(df_cleaned)

       age  balance  campaign  pdays  previous  last_contact  job_blue_collar  \
0       58     2143         1     -1         0           197                0   
2       33        2         1     -1         0           197                0   
11      29      390         1     -1         0           197                0   
12      53        6         1     -1         0           197                0   
17      57       52         1     -1         0           197                1   
...    ...      ...       ...    ...       ...           ...              ...   
45196   25      358         1     -1         0             2                0   
45201   53      583         1    184         4             1                0   
45204   73     2850         1     40         8             1                0   
45206   51      825         3     -1         0             1                0   
45207   71     1729         2     -1         0             1                0   

       job_entrepreneur  jo

### Import statments

In [6]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

### Selecting target column

In [7]:
X= df_cleaned.drop('y_yes', axis=1)  # Replace 'target_column_name' with your target column
y = df_cleaned['y_yes']

### Pipeline options
- using SelectKbest(10)
- StandardScaler
- SVC classifier

In [8]:
model = make_pipeline(SelectKBest(k=10), StandardScaler(), SVC())
kf = KFold(n_splits=10, random_state=1024, shuffle=True)

In [9]:
predictions = []
actual_labels = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    
    predictions.extend(y_pred)
    actual_labels.extend(y_test)

In [10]:
acc = accuracy_score(predictions, actual_labels)
print(f'Accuracy: {acc}')

Accuracy: 0.8962928202721727
